In [10]:
# excercise Variation Auto Encoder
# the goal is to train on Jusitn/Abbigale/Jessica portraits to establish a vector sample distribution
# and sampling from the distribution to create machine generated image.
# those image should have mixed facial characteristic of 3 peoples.
# if possible using distribution sampled vector as input to train a GAN model to create more realistic 
# images. but this step will be implemented in other file.

from tensorflow.keras import layers, Input
from tensorflow.keras.models import Model
from tensorflow.keras.backend import int_shape 
import tensorflow.keras.backend as K

import numpy as np
import tensorflow as tf

from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

#portrait_shape = (500, 400, 3)
portrait_shape = (28, 28, 1)
batch_size = 16
#latent_dim = 256
latent_dim = 2

input_img = Input(shape=portrait_shape)
x = layers.Conv2D(32, 3, padding = 'same', activation = 'relu')(input_img) #out shape: batch_size, 500,400
x = layers.Conv2D(64, 3, padding = 'same', activation = 'relu', strides = (2,2))(x) #out shape: batch_size, 250, 200
x = layers.Conv2D(64, 3, padding = 'same', activation = 'relu')(x)
x = layers.Conv2D(64, 3, padding = 'same', activation = 'relu')(x)
shape_before_flatting = int_shape(x) #shape: batch_size, 250, 200, 64

x = layers.Flatten()(x)
x = layers.Dense(32, activation = 'relu')(x)

z_mean = layers.Dense(latent_dim)(x)
z_log_var = layers.Dense(latent_dim)(x)

In [11]:
import tensorflow.keras.backend as K

def ZSampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0., stddev=1.)
    zi = z_mean + K.exp(z_log_var) * epsilon
    print('ZSampleing output:', zi)
    return zi

z = layers.Lambda(ZSampling)([z_mean, z_log_var])

ZSampleing output: Tensor("lambda_1/add:0", shape=(None, 2), dtype=float32)


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import matplotlib.pyplot as plt

def loadImage(path):
    img2 = cv2.imread(path, cv2.IMREAD_COLOR)
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
    img2 = cv2.resize(img2, (400, 500))
    
    return img2[np.newaxis,:,:,:]

def loadTrainTensor():
    path = '/Users/justin/projects/autoencoder/20210208_073907.jpg'
    data = loadImage(path).copy()
    
    path = '/Users/justin/projects/autoencoder/2021-02-08_07-45-44.jpg'
    data = np.append(data, loadImage(path).copy(), 0)
    
    path = '/Users/justin/projects/autoencoder/20210208_074628.jpg'
    data = np.append(data, loadImage(path).copy(), 0)
    
    return data

In [12]:
# VAE decoder

decode_input = layers.Input(K.int_shape(z)[1:])
x = layers.Dense(np.prod(shape_before_flatting[1:]), activation='relu')(decode_input) #out shape: batch_size, 250x200x64
x = layers.Reshape(shape_before_flatting[1:])(x) #out shape: batch_size, 250, 200, 64
x = layers.Conv2DTranspose(32, 3, padding='same', activation='relu', strides=(2,2), name='conv2d_tran')(x) #out shape: batch_size, 500, 400, 32
x = layers.Conv2D(portrait_shape[-1], 3, padding='same', activation='sigmoid', name='conv2d_output')(x) #out shape: batch_size, 500, 400, 3

decoder = Model(decode_input, x, name='decoder_layer')

print('z shape', z.shape)
print('z------', z)
z_decoded = decoder(z)


z shape (None, 2)
z------ Tensor("lambda_1/add:0", shape=(None, 2), dtype=float32)


In [13]:
# to build a custom keras layer for this complicated loss function
class CustomVariationLayer(layers.Layer):
    def vae_loss(self, x, x_decoded):
        x = K.flatten(x)
        x_decoded = K.flatten(x_decoded)
        
        xent_loss = tf.keras.metrics.binary_crossentropy(x, x_decoded)
        kl_loss = -5e-4*K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        
        return K.mean(xent_loss + kl_loss)
    
    def call(self, inputs):
        x = inputs[0]
        z_decoded = inputs[1]
        loss = self.vae_loss(x, z_decoded)
        self.add_loss(loss)
        
        return x

y = CustomVariationLayer()([input_img, z_decoded])
        

In [14]:
from tensorflow.keras.datasets import mnist

vae = Model(input_img, y)
vae.compile(optimizer='rmsprop', loss=None)
vae.summary()
decoder.summary()

(x_train, _), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_train = x_train.reshape(x_train.shape + (1,))

x_test = x_test.astype('float32') / 255.
x_test = x_test.reshape(x_test.shape + (1,))

vae.fit(x = x_train, y=None, shuffle = True, epochs = 10, batch_size=16, validation_data=(x_test, None))

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 28, 28, 32)   320         input_4[0][0]                    
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 14, 14, 64)   18496       conv2d_4[0][0]                   
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 14, 14, 64)   36928       conv2d_5[0][0]                   
____________________________________________________________________________________________

KeyboardInterrupt: 

In [ ]:

train = loadTrainTensor()
train_gen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=100, height_shift_range=80, fill_mode='nearest')
train = loadTrainTensor()

print(train_gen.mean)
train_gen.fit(train)
print(train_gen.mean)

xlist = train_gen.flow(train, train)
print(len(xlist))
print(xlist.n, xlist.__class__)
print(xlist[0][0][:1].shape)
print(xlist[0][0][1:].shape)

cnt=0
#for xe in xlist:print(cnt);cnt +=1

In [9]:
# training VAE model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau
from datetime import datetime

vae = Model(input_img, y)
vae.compile(optimizer='rmsprop', loss=None)
vae.summary()
decoder.summary()

epoches = 20

train = loadTrainTensor()
train_gen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=100, height_shift_range=80, fill_mode='nearest')

tensor_board_callback = TensorBoard(
    log_dir='/Users/justin/projects/autoencoder/tensorboard/logs' + datetime.now().strftime('%m_%d_%YT%H_%M'),
    histogram_freq=1,
    update_freq = 'epoch',
    embeddings_freq = 1)

for i in range(epoches): 
    print('cycle#', i)    
    
    cnt = 0
    for x_batch, _ in train_gen.flow(train, train):
        #print(x_batch.shape)
        #fig, ax = plt.subplots(1,3, figsize=(400, 500)) 
        #for i, ax in enumerate(ax):
        #    ax.imshow(x_batch[i])
        #print(x_batch[0])
        cnt += 1
        print('----', cnt)
        vae.fit(x = x_batch[1:], y=None, shuffle = False, batch_size=2, validation_data=(x_batch[:1], None), callbacks=[tensor_board_callback])
        

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 500, 400, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 500, 400, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 250, 200, 64) 18496       conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 250, 200, 64) 36928       conv2d_1[0][0]                   
____________________________________________________________________________________________

InvalidArgumentError: You must feed a value for placeholder tensor 'input_3' with dtype float and shape [?,256]
	 [[{{node input_3}}]]

In [ ]:
help(vae.fit)

In [ ]:
train = loadTrainTensor()

fig, ax = plt.subplots(1,3, figsize=(400, 500)) 
for i, ax in enumerate(ax):
    ax.imshow(train[i])

In [ ]:
import tensorflow.keras.backend as K

x = K.variable(np.eye(2, 2)) 

print('shape:', K.shape(x))
print('int_shape:', K.int_shape(x))
print('x.shape', x.shape)

In [ ]:
a = np.array([0,1,2,3,4])
print(a[:1].shape)
print(a[1:].shape)